In [26]:
import tensorflow as tf
import os

vocab_file = r'.\deep_learn\jd_deep_learn\jd_vocab.txt'

class Vocab:
    def __init__(self, filename, num_word_threshold):
        self._word_to_id = {}
        self._unk = -1
        self._num_word_threshold = num_word_threshold
        self._read_dict(filename)
        
    def _read_dict(self, filename):
        with open(filename, 'r', encoding='utf-8') as f:
            lines = f.readlines()
        for line in lines:
            word, frequency = line.strip('\r\n').split('\t')
            frequency = int(frequency)
            if frequency < self._num_word_threshold:
                continue
            idx = len(self._word_to_id)
            if word == '<UNK>':
                self._unk = idx
            self._word_to_id[word] = idx
    def word_to_id(self, word):
        return self._word_to_id.get(word, self._unk)
    
    @property
    def unk(self):
        return self._unk
    
    def size(self):
        return len(self._word_to_id)
    
    def sentence_to_id(self, sentence):
        
        word_ids = [self.word_to_id(cur_word) for cur_word in sentence.split()]
        return word_ids
vocab = Vocab(vocab_file, 10)
vocab_size = vocab.size()


sentence1 = '首先和评论手机，手机质感和流畅的还是很不错，系统很好就是新系统bug有点多，总体评价来说还是很不错的一个购物外形外观延用了苹果手机的一惯外形和材质，拿在手里很有质感和重量，去实体门店看过了，经常断货，没想到京东有，就果断拍下了！择这款手机的主要原因是它的 屏幕介于苹果11PRO和11PRO MAX 之间，对于我来讲，简直是为我定制的一样，怎样挑剔的我，都无话可说了！屏幕的分辨率也是很棒的，音效我感觉比以前的苹果手机也有进步。拍照效果一如既往的好啊，像素高的超前，还增加了广角拍摄，十分满意！运行速度流畅，反应速度很快，完美！'
sentence2 = '非常差，太差了，垃圾手机'
import jieba
import numpy as np
sentence_list = jieba.cut(sentence1)
words = ' '.join([word for word in sentence_list])
id_words = vocab.sentence_to_id(words)
id_words = id_words[0: 120]
padding_num = 120 - len(id_words)
id_words = id_words + [0 for _ in range(padding_num)]
np_id_words = np.asarray([id_words])
print(np_id_words.shape)
id_words_list = []
for _ in range(100):
    id_words_list.append(id_words)
# print(np.asarray(id_words_list, dtype=np.int32))
# print(np.asarray(id_words_list, dtype=np.int32).shape)
word_np = np.asarray(id_words_list, dtype=np.int32)


(1, 120)


In [14]:
def load_model(model_path):
    if not os.path.exists(model_path):
        raise ValueError("model is not exists")
    model_graph = tf.Graph()
    with model_graph.as_default():
        od_graph_def = tf.GraphDef()
        with tf.gfile.GFile(model_path, 'rb') as gf:
            serialized_data = gf.read()
            od_graph_def.ParseFromString(serialized_data)
            tf.import_graph_def(od_graph_def, name='')
    return model_graph


    

fp_model_path = r'.\deep_learn\jd_deep_learn\jd_lstm_runout\lstm_model\jd_comment_lstm_model.pb'
model_path = r'.\deep_learn\jd_deep_learn\jd_lstm_runout\model\jd_comment_cnn_serving\saved_model.pb'
model_graph = load_model(fp_model_path)
inputs_x = model_graph.get_tensor_by_name('Placeholder:0')
outputs = model_graph.get_tensor_by_name('metrics/ArgMax:0')
keep_prob = model_graph.get_tensor_by_name('keep_prob:0')
with model_graph.as_default():
    with tf.Session(graph=model_graph) as sess:
        print(inputs_x)
        for i in model_graph.as_graph_def().node:
            print(i.name)
        pred = sess.run(outputs, feed_dict={inputs_x: word_np, keep_prob:1})
        print(pred[0])


Tensor("Placeholder:0", shape=(100, 120), dtype=int32)
Placeholder
keep_prob
embedding/embedding
embedding/embedding/read
embedding/embedding_lookup/axis
embedding/embedding_lookup
embedding/embedding_lookup/Identity
lstm_nn/MultiRNNCellZeroState/DropoutWrapperZeroState/BasicLSTMCellZeroState/Const
lstm_nn/MultiRNNCellZeroState/DropoutWrapperZeroState/BasicLSTMCellZeroState/Const_1
lstm_nn/MultiRNNCellZeroState/DropoutWrapperZeroState/BasicLSTMCellZeroState/concat/axis
lstm_nn/MultiRNNCellZeroState/DropoutWrapperZeroState/BasicLSTMCellZeroState/concat
lstm_nn/MultiRNNCellZeroState/DropoutWrapperZeroState/BasicLSTMCellZeroState/zeros/Const
lstm_nn/MultiRNNCellZeroState/DropoutWrapperZeroState/BasicLSTMCellZeroState/zeros
lstm_nn/MultiRNNCellZeroState/DropoutWrapperZeroState/BasicLSTMCellZeroState/Const_4
lstm_nn/MultiRNNCellZeroState/DropoutWrapperZeroState/BasicLSTMCellZeroState/Const_5
lstm_nn/MultiRNNCellZeroState/DropoutWrapperZeroState/BasicLSTMCellZeroState/concat_1/axis
lstm_nn/M

ValueError: Cannot feed value of shape (1, 120) for Tensor 'Placeholder:0', which has shape '(100, 120)'

In [28]:
from tensorflow import saved_model
with tf.Session(graph=tf.Graph()) as sess:
    MetaGraphDef = saved_model.loader.load(sess,[tf.saved_model.tag_constants.SERVING],
                                           export_dir=".\deep_learn\jd_deep_learn\jd_lstm_runout\jd_comment_lstm_serving")
    SignatureDef_d = MetaGraphDef.signature_def
    SignatureDef = SignatureDef_d['jd_comment_lstm_predict']
    # graph = tf.get_default_graph()
    
    # x = sess.graph.get_tensor_by_name("Placeholder:0")
    # y = sess.graph.get_tensor_by_name("metrics/ArgMax:0")
    # keep_prob2 = sess.graph.get_tensor_by_name('keep_prob:0')
    x_TensorInfo = SignatureDef.inputs['inputs']
    keep_prob2_TensorInfo = SignatureDef.inputs['keep_prob']
    y_TensorInfo = SignatureDef.outputs['outputs']
    print(x_TensorInfo)
    x = saved_model.utils.get_tensor_from_tensor_info(x_TensorInfo, sess.graph)
    keep_prob2 = saved_model.utils.get_tensor_from_tensor_info(keep_prob2_TensorInfo, sess.graph)
    y = saved_model.utils.get_tensor_from_tensor_info(y_TensorInfo, sess.graph)
    
    pred = sess.run(y, feed_dict={x: word_np, keep_prob2:1})
    print(pred)

INFO:tensorflow:Restoring parameters from .\deep_learn\jd_deep_learn\jd_lstm_runout\jd_comment_lstm_serving\variables\variables
name: "Placeholder:0"
dtype: DT_INT32
tensor_shape {
  dim {
    size: 100
  }
  dim {
    size: 120
  }
}

[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]


In [31]:
from tensorflow import saved_model
with tf.Session(graph=tf.Graph()) as sess:
    MetaGraphDef = saved_model.loader.load(sess,[tf.saved_model.tag_constants.SERVING],
                                           export_dir=".\deep_learn\jd_deep_learn\jd_cnn_runout\jd_comment_cnn_serving")
    SignatureDef_d = MetaGraphDef.signature_def
    SignatureDef = SignatureDef_d['jd_comment_cnn_predict']
    # graph = tf.get_default_graph()
    
    # x = sess.graph.get_tensor_by_name("Placeholder:0")
    # y = sess.graph.get_tensor_by_name("metrics/ArgMax:0")
    # keep_prob2 = sess.graph.get_tensor_by_name('keep_prob:0')
    x_TensorInfo = SignatureDef.inputs['inputs']
    keep_prob2_TensorInfo = SignatureDef.inputs['keep_prob']
    y_TensorInfo = SignatureDef.outputs['outputs']
    print(x_TensorInfo)
    x = saved_model.utils.get_tensor_from_tensor_info(x_TensorInfo, sess.graph)
    keep_prob2 = saved_model.utils.get_tensor_from_tensor_info(keep_prob2_TensorInfo, sess.graph)
    y = saved_model.utils.get_tensor_from_tensor_info(y_TensorInfo, sess.graph)
    
    pred = sess.run(y, feed_dict={x: word_np, keep_prob2:1})
    print(pred)


RuntimeError: MetaGraphDef associated with tags 'serve' could not be found in SavedModel. To inspect available tag-sets in the SavedModel, please use the SavedModel CLI: `saved_model_cli`